### Preprocessing

In [87]:
import pickle

with open('embedding_labels.pkl', 'rb') as lf:
    labels = pickle.load(lf)

with open('embedding_posts.pkl', 'rb') as pf:
    posts = pickle.load(pf)
    
with open('posts_per_user.pkl', 'rb')as tpf:
    posts_per_user = pickle.load(tpf)

In [138]:
import json
dataFile = open('formatted_data.json', 'r')
data = []
for line in dataFile:
    data.append(json.loads(line))
    
types = []

for i in range(1, len(data)):
    types.append(data[i]['Type'])

In [88]:
# Tokenizing data
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

max_features = 10000
max_len = 50
batch_size = 32
test_samples = 68965
max_words = 10000

In [11]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(posts)
sequences = tokenizer.texts_to_sequences(posts)
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

Found 116582 unique tokens


In [139]:
data = sequence.pad_sequences(sequences, maxlen=max_len)
label_dictionary = {
    'I': 0,
    'E': 1
}

types = [label_dictionary[label[:1]] for label in types]

In [94]:
import numpy as np
labels = np.asarray(labels)

In [95]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (396704, 50)
Shape of label tensor: (396704,)


In [96]:
x_test = data[:test_samples]
y_test = labels[:test_samples]
x_train = data[test_samples:]
y_train = labels[test_samples:]

In [97]:
indices = np.arange(x_train.shape[0])
np.random.shuffle(indices)
x_train = x_train[indices]
y_train = y_train[indices]

In [102]:
from keras import models
from keras import layers
from keras.layers import Embedding, Flatten, LSTM

model = models.Sequential()
model.add(Embedding(max_features, max_len))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

# model.layers[0].set_weights([embedding_matrix])
# model.layers[0].trainable = False

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

model.fit(x_train, y_train, epochs=1, batch_size=8192)

Epoch 1/1
327739/327739 [==============================] - 75s 228us/step - loss: 0.5673 - acc: 0.7575


In [238]:
from pprint import pprint

def get_accuracy(posts_per_user, x_test, ptype):
    start_index = 0
    end_index = 28
    correct = []
    for i in range(0, 1500):
        predictions = model.predict(x_test[start_index:end_index])
        predictions = np.asarray(predictions).round()
        start_index = end_index
        end_index += posts_per_user[i+1]
        
        counter = 0
        for pred in predictions:
            counter += int(pred)
        
        personality = round((counter/(end_index-start_index))
        
        if personality == :
            correct.append(1)
        else:
            correct.append(0)
    
    return correct

SyntaxError: invalid syntax (<ipython-input-238-1ff0e7a5f597>, line 19)

In [236]:
get_accuracy(posts_per_user, x_test, types)/len(types)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.020833333333333332
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0

0.13360230547550433